# Parallelized Global Ligand Docking with `pyrosetta.distributed`

 Presenter: Jason Klima (klimaj@uw.edu)

In [1]:
import logging
logging.basicConfig(level=logging.INFO)
import json
import pandas as pd
import matplotlib
%matplotlib inline
import os
import py3Dmol
import pyrosetta
import pyrosetta.distributed.dask
import pyrosetta.distributed.io as io
import pyrosetta.distributed.packed_pose as packed_pose
import pyrosetta.distributed.tasks.rosetta_scripts as rosetta_scripts
import pyrosetta.distributed.tasks.score as score
import seaborn
seaborn.set()

from dask_jobqueue import SLURMCluster
from dask.distributed import Client, LocalCluster, progress
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/Users/andrew/anaconda3/envs/pyrosetta/lib/python3.6/site-packages/dask_jobqueue/config.py:12: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


 Setup PyRosetta command line flags:

In [2]:
ligand_params = os.path.join(os.getcwd(), "input/ATP.am1-bcc.fa.params")
flags = f"""
-extra_res_fa {ligand_params}
-ignore_unrecognized_res 1
-out:level 200
"""
pyrosetta.distributed.dask.init_notebook(flags)

INFO:pyrosetta.distributed:maybe_init performing pyrosetta initialization: {'extra_options': '-extra_res_fa /Users/andrew/rosetta/pyrosetta_code_school/apl_workthrough/source/Session20_Parallelizing_Code/input/ATP.am1-bcc.fa.params -ignore_unrecognized_res 1 -out:level 200'}
INFO:pyrosetta.rosetta:Found rosetta database at: /Users/andrew/anaconda3/envs/pyrosetta/lib/python3.6/site-packages/pyrosetta/database; using it....
INFO:pyrosetta.rosetta:PyRosetta-4 2019 [Rosetta PyRosetta4.conda.mac.python36.Release 2019.19+release.5adc612fd9dee20f808a07e761610d95698b0f35 2019-05-10T09:04:00] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.


PyRosetta-4 2019 [Rosetta PyRosetta4.conda.mac.python36.Release 2019.19+release.5adc612fd9dee20f808a07e761610d95698b0f35 2019-05-10T09:04:00] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.


## Whether you are running this Jupyter Notebook example on laptop/desktop or high-performance computing (HPC) cluster, run one of the following cells:

 1. If you are running on a laptop/desktop:

In [3]:
cluster = LocalCluster(n_workers=4, threads_per_worker=1)
client = Client(cluster)
client.run(pyrosetta.distributed.dask.init_notebook, flags)

{'tcp://127.0.0.1:52498': None,
 'tcp://127.0.0.1:52500': None,
 'tcp://127.0.0.1:52501': None,
 'tcp://127.0.0.1:52504': None}

 2. If you are running this on a high-performance computing (HPC) cluster with SLURM scheduling, uncomment and run the following cell to setup `dask` workers to process docking simulations:

In [ ]:
# scratch_dir = os.path.join("/net/scratch", os.environ["USER"])
# cluster = SLURMCluster(cores=1,
#                        processes=1,
#                        job_cpu=1,
#                        memory="3GB",
#                        queue="interactive",
#                        local_directory=scratch_dir,
#                        job_extra=["-o {}".format(os.path.join(scratch_dir, "slurm-%j.out"))],
#                        extra=pyrosetta.distributed.dask.worker_extra(init_flags=flags))
# cluster.scale(20) # Instead of `cluster.scale(20)`, could use `cluster.adapt(minimum=0, maximum=20)` to automatically scale number of workers to amount of work load
# client = Client(cluster)

In [4]:
client

Client Scheduler: tcp://127.0.0.1:52493 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 17.18 GB


 Setup global ligand docking RosettaScripts protocol within `pyrosetta.distributed`:

In [5]:
xml = """
<ROSETTASCRIPTS>
  <SCOREFXNS>
    <ScoreFunction name="fa_standard" weights="ref2015.wts"/>
  </SCOREFXNS>
  <RESIDUE_SELECTORS>
    <Chain name="chX" chains="X"/>
  </RESIDUE_SELECTORS>
  <SIMPLE_METRICS>
    <RMSDMetric name="rmsd_chX" residue_selector="chX" reference_name="store_native" residue_selector_ref="chX" robust="true" rmsd_type="rmsd_all" />
  </SIMPLE_METRICS>
  <SCORINGGRIDS ligand_chain="X" width="25">
    <ClassicGrid grid_name="vdw" weight="1.0"/>
  </SCORINGGRIDS>
  <LIGAND_AREAS>
    <LigandArea name="docking_sidechain_X" chain="X" cutoff="6.0" add_nbr_radius="true" all_atom_mode="true" minimize_ligand="10"/>
    <LigandArea name="final_sidechain_X" chain="X" cutoff="6.0" add_nbr_radius="true" all_atom_mode="true"/>
    <LigandArea name="final_backbone_X" chain="X" cutoff="7.0" add_nbr_radius="false" all_atom_mode="true" Calpha_restraints="0.3"/>
  </LIGAND_AREAS>
  <INTERFACE_BUILDERS>
    <InterfaceBuilder name="side_chain_for_docking" ligand_areas="docking_sidechain_X"/>
    <InterfaceBuilder name="side_chain_for_final" ligand_areas="final_sidechain_X"/>
    <InterfaceBuilder name="backbone" ligand_areas="final_backbone_X" extension_window="3"/>
  </INTERFACE_BUILDERS>
  <MOVEMAP_BUILDERS>
    <MoveMapBuilder name="docking" sc_interface="side_chain_for_docking" minimize_water="true"/>
    <MoveMapBuilder name="final" sc_interface="side_chain_for_final" bb_interface="backbone" minimize_water="true"/>
  </MOVEMAP_BUILDERS>
  <MOVERS>
    <SavePoseMover name="spm" restore_pose="0" reference_name="store_native"/>
    <Transform name="transform" chain="X" box_size="20.0" move_distance="10" angle="360" initial_perturb="2" cycles="500" repeats="5" temperature="1000"/>
    <HighResDocker name="high_res_docker" cycles="9" repack_every_Nth="3" scorefxn="fa_standard" movemap_builder="docking"/>
    <FinalMinimizer name="final" scorefxn="fa_standard" movemap_builder="final"/>
  </MOVERS>
  <FILTERS>
      <LigInterfaceEnergy name="interfE" scorefxn="fa_standard" energy_cutoff="0.0" confidence="0"/>
      <SimpleMetricFilter name="rmsd_chX" metric="rmsd_chX" cutoff="999999." comparison_type="lt" confidence="0"/>
  </FILTERS>
  <PROTOCOLS>
    <Add mover="spm"/>
    <Add mover="transform"/>
    <Add mover="high_res_docker"/>
    <Add mover="final"/>
    <Add filter="interfE"/>
    <Add filter="rmsd_chX"/>
  </PROTOCOLS>
</ROSETTASCRIPTS>
"""
xml_obj = rosetta_scripts.SingleoutputRosettaScriptsTask(xml)
xml_obj.setup()

 Setup input pose as `PackedPose` object:

In [6]:
pdbfile = os.path.join(os.getcwd(), "input/test_lig.pdb")
pose_obj = io.pose_from_file(filename=pdbfile)w

 Submit 100 global ligand docking trajectories, very similarly to using the command line `-nstruct` flag:

In [7]:
futures = [client.submit(xml_obj, pose_obj) for i in range(100)]
results = [future.result() for future in futures]

 Calling `future.result()` transfers the `PackedPose` objects back to this Jupyter session, so we can inspect the scores!

In [ ]:
df = pd.DataFrame.from_records(packed_pose.to_dict(results))
df.head(10)

 Now plot the ligand binding energy landscape:

In [ ]:
matplotlib.rcParams['figure.figsize'] = [12.0, 8.0]
seaborn.scatterplot(x="rmsd_chX", y="interfE", data=df)

 Let's view the lowest `interfE` ligand dock:

In [ ]:
%run ./visualization.py
view_pose(list(packed_pose.dict_to_pose(df.sort_values(by="interfE").head(1).to_dict()).values())[0])

 Let's view the 5 lowest `interfE` ligand docks:

In [ ]:
view_poses(list(packed_pose.dict_to_pose(df.sort_values(by="interfE").head(5).to_dict()).values())) 

 If you wish to save any `PackedPose` objects as `.pdb` files:

In [ ]:
# for i, p in enumerate(results):
#     with open("RESULT_%i.pdb" % i, "w") as f:
#         f.write(io.to_pdbstring(p))

 If you wish to save a scorefile:

In [ ]:
with open(os.path.join(os.getcwd(), "scores.fasc"), "w") as f:
    for result in results:
        json.dump(result.scores, f)